#### Data

In [334]:
# libraries
import numpy as np
import pandas as pd
import altair as alt
alt.renderers.enable('default')

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import add_dummy_feature

# warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# display settings
pd.options.display.max_columns = None

In [335]:
# load tidied data and print rows
data = pd.read_csv(
    'tidy-data', 
    dtype = {'pop_trans_acc':'Int64',
             'county_fips': 'Int64'},
    index_col = 0
)

data.head()

,year,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips,region_name,region_code,pop_trans_acc,pop2010,p_trans_acc,LL_95CI,UL_95CI,se,rse
0,2008,3,AfricanAm,CO,6061,Placer,Placer,6061,Sacramento Area,8,55,4427,0.012424,0.009161,0.015687,0.001665,13.399974
1,2008,1,AIAN,CO,6061,Placer,Placer,6061,Sacramento Area,8,51,2080,0.024519,0.017873,0.031166,0.003391,13.830066
2,2008,2,Asian,CO,6061,Placer,Placer,6061,Sacramento Area,8,117,19963,0.005861,0.004802,0.006920,0.000540,9.217872
3,2008,4,Latino,CO,6061,Placer,Placer,6061,Sacramento Area,8,1835,44710,0.041042,0.039203,0.042881,0.000938,2.286029
4,2008,7,Multiple,CO,6061,Placer,Placer,6061,Sacramento Area,8,241,10658,0.022612,0.019790,0.025435,0.001440,6.368321


#### Exploratory

**County Data**

In [336]:
# county-level data, do not include 'Total' in race_eth_name
data_county = data[(data.geotype == 'CO') & (data.race_eth_name != 'Total')]

# merge the pop2010 and p_trans_acc statistics into one dataframe 
county_pop_percent = pd.merge(data_county.groupby(['county_name']).sum().reset_index().loc[:, ['county_name', 'pop2010']], 
                              data_county.groupby(['county_name']).mean().reset_index().loc[:, ['county_name', 'p_trans_acc']])

# drop counties with '0' for 'p_trans_acc'
county_pop_percent = county_pop_percent[(county_pop_percent['p_trans_acc'] != 0)]

In [337]:
# scatter plot of counties
scatter_county = alt.Chart(county_pop_percent).mark_point().encode(
    x = alt.X('pop2010',
              title = '2010 County Population',
              scale = alt.Scale(zero = False, type = 'pow', exponent = 0.5)),
    y = alt.Y('mean(p_trans_acc)', 
              title = 'Percent Access')
).properties(
    title = 'Access to Public Transportation by County',
    width = 700, 
    height = 275
)

# labels 
text_CO = scatter_county.mark_text(
    align = 'left',
    baseline ='middle',
    dx = 5,
    dy = 0.2
).encode(
    text='county_name'
)

# plot
county_plot = scatter_county + text_CO
county_plot

alt.LayerChart(...)

**Regional Data**

In [338]:
# county-level data, do not include 'Total' in race_eth_name
data_region = data[((data.geotype == 'RE') | ((data.region_name == 'San Diego') & (data.geotype == 'CO'))) & 
                   (data.race_eth_name != 'Total')]

# merge the pop2010 and p_trans_acc statistics into one dataframe 
region_pop_percent = pd.merge(data_region.groupby(['region_name']).sum().reset_index().loc[:, ['region_name', 'pop2010']], 
                              data_region.groupby(['region_name']).mean().reset_index().loc[:, ['region_name', 'p_trans_acc']])

# renaming the columns
region_pop_percent = region_pop_percent.rename(
    columns = {'region_name':'Region', 'pop2010': '2010 Population', 'p_trans_acc': 'Percent Access'})

region_pop_percent

,Region,2010 Population,Percent Access
0,Bay Area,7150739,0.571932
1,Sacramento Area,1999270,0.190002
2,San Diego,3095313,0.358582
3,Southern California,18051534,0.351986


**Access by Region and Ethnicity**

In [339]:
# bar chart 
alt.Chart(
    data_region
).mark_bar().encode(
    x = alt.X('race_eth_name', 
              title = ''), 
    y = alt.Y('mean(p_trans_acc)', 
              scale=alt.Scale(domain=[0, 0.8]), 
              title = 'Percent Access')
).properties(
    width = 250, height = 250
).facet(
    column = 'region_name').properties(title = 'Access to Public Transportation by Ethnicity & Region')

alt.FacetChart(...)

#### Modeling the Data

**Can we predict the access to public transportation based on the population of each county?**

In [340]:
## Simple Linear Regression

# store response variable as array
y = county_pop_percent["p_trans_acc"]

# store explanatory variable
x_slr_df = county_pop_percent.loc[: , ['pop2010']]

# add intercept column
x_slr = add_dummy_feature(x_slr_df, value = 1)

# configure model
slr = LinearRegression(fit_intercept = False)

# fit model
slr.fit(x_slr, y)

# fitted values
fitted_slr = slr.predict(x_slr)

# residuals
resid_slr = y - fitted_slr

# append fitted values and residuals to the data
county_pop_percent_slr = county_pop_percent.copy()
county_pop_percent_slr['fitted_slr'] = fitted_slr
county_pop_percent_slr['resid_slr'] = resid_slr

# compute R-squared
r2_score(county_pop_percent_slr.p_trans_acc, county_pop_percent_slr.fitted_slr)

0.021445586694845753

The $R^2$ value is insignificant because it is roughly 0.

In [341]:
# modifying scale from original 'scatter_county' plot
scatter_county_mod1 = scatter_county.encode(
    x = alt.X('pop2010', title = '2010 County Population')
    ).properties(title = 'Public Transportation by County with Simple Regression Line')

# modifying labels from original 'scatter_county' plot
text_CO_mod1 = scatter_county_mod1.mark_text(
    align = 'left', 
    baseline ='middle', 
    dx = 5, 
    dy = 0.2
).encode(text='county_name')

# plot
county_plot_mod1 = scatter_county_mod1 + text_CO_mod1

# adding the line of best fit
slr_line = alt.Chart(
    county_pop_percent_slr
).mark_line(
).encode(
    x = 'pop2010',
    y = 'fitted_slr'
)
 
# layer
county_plot_mod1 + slr_line

alt.LayerChart(...)

 A simple linear regression model does not fit the data well.

**Examining the demographics in each county**

In [342]:
# make a dataframe with 'race_eth_name', 'county_name', 'pop2010 for the ethicity', total county population
data_county_mlr = data_county.groupby(['county_name', 'race_eth_name']).sum().reset_index().loc[:, ['race_eth_name', 'county_name', 'pop2010']]


# merging, in order to get total population values from 'county_pop_percent' data set
data_county_mlr = pd.merge(
    data_county_mlr,
    county_pop_percent.drop(columns = 'p_trans_acc').rename(columns = {'pop2010':'county_total'}), 
    how='left'
    )

# percent of ethnicity in each county
data_county_mlr['race_eth_percent'] = data_county_mlr.pop2010 / data_county_mlr.county_total

# pivot table to make the columns a percent of each race 
data_county_mlr = data_county_mlr.pivot_table('race_eth_percent', ['county_name'], 'race_eth_name').reset_index().rename_axis(None, axis = 1)

# meging, in order to get 'p_trans_acc' from 'county_pop_percent' data set
data_county_mlr = pd.merge(
    data_county_mlr,
    county_pop_percent,
    how='left'
    )

# sorting the values by descending rates to public transit
data_county_mlr = data_county_mlr.sort_values(by = 'p_trans_acc', ascending = False)

data_county_mlr.head()

,county_name,AIAN,AfricanAm,Asian,Latino,Multiple,NHOPI,Other,White,pop2010,p_trans_acc
11,San Francisco,0.002270,0.058096,0.329966,0.151228,0.032387,0.003885,0.003097,0.419071,805235,0.992901
13,Santa Clara,0.002269,0.023760,0.317385,0.268971,0.030059,0.003509,0.002176,0.351871,1781642,0.673294
0,Alameda,0.002774,0.121916,0.258579,0.225052,0.040299,0.007900,0.002775,0.340706,1510271,0.659370
12,San Mateo,0.001566,0.026116,0.244880,0.254021,0.033301,0.013757,0.003771,0.422588,718451,0.576735
2,Los Angeles,0.001923,0.083014,0.135016,0.477450,0.019852,0.002288,0.002584,0.277873,9818605,0.528544


In [343]:
alt.Chart(
    data_county_mlr
).mark_bar().encode(
    x = alt.X('county_name', 
              title = ''), 
    y = alt.Y('p_trans_acc', 
              title = 'Percent Access to Transit'),
    color = 'AfricanAm'
).properties(
    width = 300, height = 250, title = 'African American'
)

alt.Chart(...)

#### Multiple Linear Regression

In [344]:
# categorical variable encoding
x_df = data_county_mlr[['White', 'pop2010']]

# add intercept 
x_mx = add_dummy_feature(x_df, value = 1)

# response variable
y_mlr = data_county_mlr['p_trans_acc']

# configure model
mlr = LinearRegression(fit_intercept = False)

# fit model
mlr.fit(x_mx, y_mlr)

# fitted values
fitted_mlr = mlr.predict(x_mx)

# residuals
resid_mlr = y_mlr - fitted_mlr

In [345]:
# use ".copy()" to solve potential error warning
data_county_mlr_copy = data_county_mlr.copy()

# append fitted and residual values
data_county_mlr_copy.loc[: , "fitted_mlr"] = fitted_mlr
data_county_mlr_copy.loc[: , "resid_mlr"] = resid_mlr

# R^2 value
R_2_mlr = r2_score(data_county_mlr_copy.p_trans_acc, data_county_mlr_copy.fitted_mlr)
R_2_mlr

0.11633862102024639

The $R^2$ improves substantially from simple linear regression by adding an ethnicity ('White') as another parameter. Adding more parameters, continues to increase the $R^2$ value. For example, adding either 'Asian' or 'Latino' in addition to 'White', increases the $R^2$ value to ~ 70%. However, this is because **the model is overfitting the data** and the sample size is small. Nonetheless, it shows that ethnicity is still important in determining transportation access rates in a county.